In [ ]:
'''
   Copyright 2023 Spacebel s.a.

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
'''

# STAC API

## Overview

This notebook explains the use of the STAC API interface with GeoJSON response format. It uses the `pystac` [[RD17]](#RD17) and `pystac_client` [[RD18]](#RD18) libraries to access the interface. The visualisation of search results is borrowed from the ODC notebook available at [[RD19]](#RD19).  Examples using `curl` on the command-line are provided as well.

In [ ]:
%pip install geopandas
%pip install --force-reinstall -v "pystac_client==0.6.1"
%pip install intake

In [ ]:
%pip install folium matplotlib mapclassify
%pip install jsonpath_ng

In [ ]:
import folium
import folium.plugins
import geopandas as gpd
import shapely.geometry
import pandas as pd  
import numpy as np
import json
import urllib.parse
import requests

from xml.dom import minidom
from IPython.display import HTML, display
from IPython.display import Markdown as md
from pystac_client import Client
from pystac import Collection
from typing import Any, Dict
from urllib.parse import urlparse, parse_qsl
from matplotlib import pyplot as plt, cm, colors
from PIL import Image
from io import BytesIO
from branca.element import Figure
from concurrent.futures import ThreadPoolExecutor


def convert_bounds(bbox, invert_y=False):
    """
    Helper method for changing bounding box representation to leaflet notation

    ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))``
    """
    x1, y1, x2, y2 = bbox
    if invert_y:
        y1, y2 = y2, y1
    return ((y1, x1), (y2, x2))

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    

def curl_command( url: str, method: str = "GET" ) -> str:
    """
    Convert request URL to equivalent curl GET or POST command-line
    for STAC search (bash shell).
    """
    c = "curl -X " + method
    res = urlparse(url)
    if "GET" in method:
        c = c + " -G " + res.scheme + "://" + res.netloc + res.path
    else:
        c = c + " " + res.scheme + "://" + res.netloc + res.path    \
              + " \\\n\t--header 'Content-Type: application/json'"  \
              + " \\\n\t--data-raw '{"
    
    lst = parse_qsl(res.query)
    
    first = True 
    for i in lst:
        # print(i[0])
        # add \ to end of previous line
        if "GET" in method:
            # correction 16/3: data-urlencode used.
            c = c + ' \\\n\t--data-urlencode "'+i[0]+'='+i[1]+'"'
        else:
            if not(first):
                c = c + ','
                
            if i[0] in ["ids","collections"]:
                # "collections" and "ids") parameter has to be included as an array.
                lst = i[1].split(',')       
                c = c + '\n\t\t"'+i[0]+'": '+str(lst).replace("'","\"")
            
            elif is_number(i[1]) or i[1][0]=='{' or i[1][0]=='[':
                # do not surround with quotes if numerical value  or a json object or an array
                c = c + '\n\t\t"'+i[0]+'": '+i[1]
            else:
                c = c + '\n\t\t"'+i[0]+'": "'+i[1]+'"'
        first = False
        
    if "POST" in method:
        c = c + "\n\t}'"
    return c


def display_previews(results):
    """
    Helper method for displaying a grid of quicklooks (if available)
    """
    # create figure
    fig = plt.figure(figsize=(20, 20))
  
    # setting values to rows and column variables for the image grid
    rows = 8
    columns = 2
    pos = 1

    for item in results.items():
        # print(item.id)
        assets = item.assets
        try:
            # print("found thumbnail", assets['thumbnail'].href)
            url = assets['thumbnail'].href
            response = requests.get(url)
            Image1 = Image.open(BytesIO(response.content))
            # display at position 'pos'
            fig.add_subplot(rows, columns, pos)
            pos = pos+1
            # show the image
            plt.imshow(Image1)
            plt.axis('off')
            plt.title(item.id)
        except:
            pass
    return

def display_gdf_plot(results):
    """
    Helper method for displaying results as dataframe plot
    """
    # https://github.com/opendatacube/odc-stac (License Apache 2.0)
    # https://odc-stac.readthedocs.io/en/latest/notebooks/stac-load-e84-aws.html#Plot-STAC-Items-on-a-Map

    # Convert STAC items into a GeoJSON FeatureCollection
    stac_json = results.get_all_items_as_dict()

    gdf = gpd.GeoDataFrame.from_features(stac_json, "epsg:4326")

    fig = gdf.plot(
        "datetime",
        edgecolor="black",
        categorical=True,
        aspect="equal",
        alpha=0.5,
        figsize=(6, 12),
        legend=True,
        legend_kwds={"loc": "upper left", "frameon": False, "ncol": 1},
    )
    _ = fig.set_title("STAC Query Results")

    # gdf
    return

def display_date_distribution(results):
    """
    Helper method for displaying number of results per year/month as bar chart
    """
    items = list(results.get_items())
    stac_json = results.get_all_items_as_dict()
    gdf = gpd.GeoDataFrame.from_features(stac_json)

    gdf['date'] = pd.to_datetime(gdf['start_datetime'])
    # create a representation of the month with strfmt
    gdf['year_month'] = gdf['date'].map(lambda dt: dt.strftime('%Y-%m'))
    grouped_df = gdf.groupby('year_month')['year_month'].size().to_frame("count").reset_index()
    grouped_df.plot(kind='bar', x='year_month', y='count')
    return

def display_value_distribution(results, column):
    """
    Helper method for displaying number values in column as bar chart
    """
    items = list(results.get_items())
    stac_json = results.get_all_items_as_dict()
    gdf = gpd.GeoDataFrame.from_features(stac_json)

    # gdf['date'] = pd.to_datetime(gdf['start_datetime'])
    # create a representation of the month with strfmt
    # gdf['year_month'] = gdf['date'].map(lambda dt: dt.strftime('%Y-%m'))

    try:
        grouped_df = gdf.groupby(column)[column].size().to_frame("count").reset_index()
        grouped_df.plot(kind='bar', x=column, y='count')
    except:
        print(column + " values are not available.")

    return


def display_map(results):
    """
    Helper method for displaying results on a map
    """
    # https://github.com/python-visualization/folium/issues/1501
    stac_json = results.get_all_items_as_dict()
    gdf = gpd.GeoDataFrame.from_features(stac_json, "epsg:4326")
    
    fig = Figure(width="800px", height="500px")
    map1 = folium.Map()
    fig.add_child(map1)

    # folium.GeoJson(
    #    shapely.geometry.box(*bbox),  # ??
    #    style_function=lambda x: dict(fill=False, weight=1, opacity=0.7, color="olive"),
    #    name="Query",
    # ).add_to(map1)

    gdf.explore(
        "start_datetime",
        categorical=True,
        tooltip=[
            "title", "datetime", "start_datetime", "platform", "instruments"    
        ],
        popup=True,
        style_kwds=dict(fillOpacity=0.1, width=2),
        name="STAC",
        m=map1,
    )

    map1.fit_bounds(bounds=convert_bounds(gdf.unary_union.bounds))
    display(fig)
    return


In [ ]:
URL_LANDING_PAGE =  'https://emc.spacebel.be/' 

In [ ]:
COLLECTION_ID1 = 'PROBA.CHRIS.1A'
COLLECTION_ID2 = 'SPOT-6.and.7.ESA.archive'  
COLLECTION_ID2_CLOUDS = 'LANDSAT.ETM.GTC'
COLLECTION_ID3_CLOUDS = 'IKONOS.ESA.archive'
COLLECTION_ID4 = 'Deimos-1.and.2.ESA.archive' 
COLLECTION_ID5_DMSMM = 'NOAA_AVHRR_L1B_LAC'  # collection with Data Mgt and Stewardship Maturity Matrix


```{index} double: STAC API ; landing page
```

### Access landing page

The landing page provides access to collections (rel="`data`"), child catalogs (rel="`child`") and the STAC item search endpoint (rel="`search`").
Get the catalogue landing page with links to other resources and available collections.

In [ ]:
curl_str = curl_command(URL_LANDING_PAGE)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
from pystac_client import Client 

api = Client.open(URL_LANDING_PAGE) 
# show as a dictionary
api.to_dict()

In [ ]:
# Get catalog title and description
print("Title\t\t:", api.title)
print("Description\t:", api.description)
print("Search link\t:", api.get_search_link())

# List child catalogs
print("Child catalogs:")
# for child in api.get_children():
#    print("\t", child.id)
    
children = [c for c in api.get_children()]
children

In [ ]:
# Show title and id for each of the children.
for count, child in enumerate(children):
    print(f'\t{count} - {child.title}, id="{child.id}"')

The collections are organised as a tree structure which can be traversed until arriving to a collection with items (granules).  Get the details of one of the children of the root catalog as an example.

In [ ]:
child = children[1]
child.to_dict()

In [ ]:
# May have again children, get the list
children = [c for c in child.get_children()]
children

In [ ]:
# Function to print subcatalogs as a tree structure.

def print_catalog_as_tree(parent, level=0, max_level=2, max_children=5, indent=""):
    """
    Helper method to show catalog hierarchy as indented list
    """
    children = [c for c in parent.get_children()]
    
    # if (level < 2) and len(children)>0:
    #   for count, child in enumerate(children):
    #        print(f'\t{indent}{count} - {child.title}, id="{child.id}"')
    #        print_catalog_as_tree(children[count], level+1, indent+"  ")
    
    n = len(children)
    if (level < max_level) and (n>0):
        i = 0
        while (i < n) and ((i < max_children) or (level==0)):
            print(f'{indent}{i} - {children[i].title}, id="{children[i].id}"')
            print_catalog_as_tree(children[i], level+1, max_level, max_children, indent+"  ")
            i = i + 1
    return 

In [ ]:
# Display shortened version of the catalog/collection tree structure.
# print_catalog_as_tree( api, max_level=3, max_children=3 )

```{index} double: pystac_client ; collection search
```
```{index} double: STAC API ; collection search
```
## Collection Search

The API implements the STAC API Collection Search Extension [[RD25]](#RD25).
Available collections can be retrieved from the landing page using a paging mechanism (with rel="`next`" links).  It requires the compliance class to be present.  As `pystac_client` does not support collection search, the `requests` library is used in the examples. 

In [ ]:
md(f"The collection endpoint is available as rel='`data`' link at JSONPath $.links[?(@.rel=='data')].  Alternatively, collections can be found by traversing the root catalog (i.e. landing page) and following the rel='`child`' links recursively.  Searchable collections have type: 'Collection' and do not have year/month/day information in their id.  E.g. `{COLLECTION_ID1}` is a searchable collection, \
while `{COLLECTION_ID1}-2022` or `{COLLECTION_ID1}-2022-12` or `{COLLECTION_ID1}-2022-12-06` cannot be used for STAC Item Search.  This limitation may be relaxed in future versions.")

In [ ]:
from jsonpath_ng.ext import parse

response = requests.get(URL_LANDING_PAGE)
data = json.loads(response.text)
expression = parse("$.links[?(@.rel == 'data')].href")
r = expression.find(data)
r[0].value

In [ ]:
# retrieve /collections response
response = requests.get(r[0].value)
data = json.loads(response.text)
jstr = json.dumps(data, indent=3)
md("```json\n" + jstr + "\n```\n")

The link with rel="http://www.opengis.net/def/rel/ogc/1.0/queryables" provides access to the list of filter criteria available for collection search.  It returns a Queryables object in JSON Schema format.

In [ ]:
from jsonpath_ng.ext import parse

expression = parse("$.links[?(@.rel == 'http://www.opengis.net/def/rel/ogc/1.0/queryables')].href")
r = expression.find(data)
r[0].value

In [ ]:
# Get queryables response and list parameters alphabetically.
response = requests.get(r[0].value)
data = json.loads(response.text)    
df = pd.DataFrame(data['properties'].items(),columns=['key','value'])
df['type'] = df.apply(lambda row : row[1]['type'], axis = 1)
df['format'] = df.apply(lambda row : row[1]['format'] if 'format' in row[1] else '-' , axis = 1)
df.drop('value',axis=1).sort_values(by=['key'])

Note: The builtin `get_collections()` function of `pystac_client` is not particularly helpful to retrieve all searchable collections.  Its behaviour depends on the presence or absense of the `collections` conformance class (https://api.stacspec.org/v1.0.0-rc.2/collections) in the landing page of the API and may therefore provide unexpected results.  When `child` links are retrieved as `collection`, this is not done recursively, and the first-level child catalogs are retrieved instead.

In [ ]:
URL_LANDING_PAGE

In [ ]:
from pystac_client import Client, ConformanceClasses 

api = Client.open(URL_LANDING_PAGE) 
api._conforms_to(ConformanceClasses.COLLECTIONS)


In [ ]:
for collection in api.get_collections():
    print(collection)

The STAC API Collection Search Extension [[RD25]](#RD25) allows retrieving the collections at the `/collections` endpoint.
List available collections using `curl`:

In [ ]:
curl_str = curl_command(URL_LANDING_PAGE+"collections")
md("```shell\n" + curl_str + "\n```\n")

### Search by free text

**Example: 1.1**  
>  Search collections by platform (`filter` and `query`).  

In [ ]:
value = 'Seasat'
params = { 'filter': "query='" + value + "'"} 
URL = f'{URL_LANDING_PAGE}collections?{urllib.parse.urlencode(params)}'

In [ ]:
curl_str = curl_command(URL)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL)
data = json.loads(response.text)
df = pd.json_normalize(data, record_path=['collections'])
df[['id', 'keywords']]

### Search by title

**Example: 1.2**  
>  Search collections by title (`filter` and `title`). 

In [ ]:
# CQL2 Basic only supports "=" operator for strings, thus complete title has to be provided.
# Future versions may support "Advanced Comparison Operators". 
value = 'ALOS PALSAR products'
params = { 'filter': "title='" + value + "'"} 
URL = f'{URL_LANDING_PAGE}collections?{urllib.parse.urlencode(params)}'

In [ ]:
curl_str = curl_command(URL)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL)
data = json.loads(response.text)
df = pd.json_normalize(data, record_path=['collections'])

df[['id', 'title']]
# df

### Search by platform

**Example: 1.3**  
>  Search collections by platform (`filter` and `platform`).  

In [ ]:
URL = URL_LANDING_PAGE + "collections"+ "?filter=platform='PROBA-1'"

In [ ]:
curl_str = curl_command(URL)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL)
data = json.loads(response.text)

for f in data['collections']:
     # use stac_client representation for collection
     c = Collection.from_dict(f)
     print(c.title)

In [ ]:
df = pd.json_normalize(data, record_path=['collections'])
df[['title', 'summaries.platform']]

### Search by organisation

**Example: 1.4**  
>  Search collections by organisation (`filter`). 

In [ ]:
URL = URL_LANDING_PAGE + "collections"+ "?filter=organisationName='ESA/ESRIN'"

In [ ]:
curl_str = curl_command(URL)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL)
data = json.loads(response.text)
# response.text

# for f in data['collections']:
#     c = Collection.from_dict(f)
      # print(type(c.providers))
      # p = list(c.providers())
#     print(c.title)
     

In [ ]:
df = pd.json_normalize(data, record_path=['collections'])
df[['title', 'providers']]

### Search by bounding box

**Example: 1.5**  
>  Search collections by bounding box (`bbox`). 

In [ ]:
URL = URL_LANDING_PAGE + "collections"+ "?bbox=14.90,37.700,14.99,37.780"

In [ ]:
curl_str = curl_command(URL)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL)
data = json.loads(response.text)
df = pd.json_normalize(data, record_path=['collections'])
df[['id', 'extent.spatial.bbox']]

### Search by temporal extent

**Example: 1.6**  
>  Search collections by temporal extent (`datetime` with closed range). 

In [ ]:
URL = URL_LANDING_PAGE + "collections"+ "?datetime=" + '2002-01-01T00:00:00.000Z/2003-12-31T23:59:59.999Z'

In [ ]:
curl_str = curl_command(URL)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL)
data = json.loads(response.text)

df = pd.json_normalize(data, record_path=['collections'])
df[['id', 'extent.temporal.interval']]

**Example: 1.7**  
>  Search collections by temporal extent (`datetime` with open range). 

In [ ]:
URL = URL_LANDING_PAGE + "collections"+ "?datetime=" + '../2001-12-31T23:59:59.999Z'

In [ ]:
curl_str = curl_command(URL)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL)
data = json.loads(response.text)

df = pd.json_normalize(data, record_path=['collections'])
df[['id', 'extent.temporal.interval']]

**Example: 1.8**  
>  Search collections by temporal extent (`datetime` with single date).

In [ ]:
URL = URL_LANDING_PAGE + "collections"+ "?datetime=" + '2002-12-31T23:59:59.999Z'

In [ ]:
curl_str = curl_command(URL)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL)
data = json.loads(response.text)

df = pd.json_normalize(data, record_path=['collections'])
df[['id', 'extent.temporal.interval']]

### Get by identifier

**Example: 1.9**  
>  Get collections by identifier (`ids`). 

In [ ]:
URL = URL_LANDING_PAGE + 'collections?ids=' + COLLECTION_ID1 + ',' + COLLECTION_ID2

In [ ]:
curl_str = curl_command(URL)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL)
data = json.loads(response.text)

df = pd.json_normalize(data, record_path=['collections'])
df[['id', 'title', 'extent.temporal.interval']]

**Example: 1.10**  
>  Get collection by identifier.  

In [ ]:
md(f"The collection metadata for `{COLLECTION_ID1}`, is available at  \
at {URL_LANDING_PAGE + 'collections/' + COLLECTION_ID1}.  This corresponds to one of the many representations available at {URL_LANDING_PAGE + 'collections/series/items/' + COLLECTION_ID1} using content-negotiation.")

In [ ]:
URL = URL_LANDING_PAGE + 'collections/' + COLLECTION_ID1

In [ ]:
curl_str = curl_command(URL)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
# retrieve collection by identifier
response = requests.get(URL)
data = json.loads(response.text)

print("id: ", data['id'])
print("title: ", data['title'])
df = pd.json_normalize(data, max_level = 0)
df.transpose()

In [ ]:
jstr = json.dumps(data, indent=3)
md("```json\n" + jstr + "\n```\n")

### Search by DOI

**Example: 1.11**  
>  Search collections by DOI (`filter` with `doi`). 

In [ ]:
value = '10.5270/esa-qoe849q'
params = { 'filter': "doi = '" + value + "'"} 
URL = f'{URL_LANDING_PAGE}collections?{urllib.parse.urlencode(params)}'

In [ ]:
curl_str = curl_command(URL)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL)
data = json.loads(response.text)

df = pd.json_normalize(data, record_path=['collections'])
df[['id', 'title']]


### Search by concept

**Example: 1.12**  
>  Search collections by concept URI (`filter` with `classifiedAs`). 


Collection metadata includes platform, instrument and science keywords, including the URI of these concepts expressed in the ESA thesauri and NASA GCMD thesauri.  The URI of these concepts can be used as search parameter.  

In the current version of the software, the following concept URIs are supported:

* ESA thesaurus platform URI
* ESA thesaurus instrument URI
* ESA thesaurus earth topic URI
* GCMD platform URI
* GCMD instrument URI
* GCMD science keyword URI

In [ ]:
# Concept defining PROBA-1
# https://gcmd.earthdata.nasa.gov/kms/concept/fe4a4604-029e-4cdc-93f0-6d8799dd25e5
# Concept defining ENVISAT
# https://gcmd.earthdata.nasa.gov/kms/concept/11ea961b-1d0b-5d6d-a55a-b58aed01d430

concept_uri = 'https://earth.esa.int/concept/b3979ff2-d27d-5f22-9e06-a18c5759d9a5'
URL = URL_LANDING_PAGE + "collections"+ "?filter=classifiedAs='" + concept_uri +  "'"

In [ ]:
curl_str = curl_command(URL)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL)
data = json.loads(response.text)

df = pd.json_normalize(data, record_path=['collections'])
df[['id', 'title']]

In [ ]:
# Get more details about the ESA Thesauri concept via the SPARQL interface.
# Make SPARQL request to obtain concept details.
#q="DESCRIBE <" + concept_uri + "> WHERE { }"
#response = requests.post(
#    'https://eovoc.spacebel.be/thesaurus/sparql', 
#    data=q, headers={'content-type': 'application/sparql-query', 'Accept': 'application/ld+json'})
# Can also use application/rdf+xml
#jstr = response.text
#md("```json\n" + jstr + "\n```\n")


In [ ]:
# Concept defining PROBA-1
concept_uri = 'https://gcmd.earthdata.nasa.gov/kms/concept/fe4a4604-029e-4cdc-93f0-6d8799dd25e5'
# ENVISAT: 11ea961b-1d0b-5d6d-a55a-b58aed01d430
URL = URL_LANDING_PAGE + "collections"+ "?filter=classifiedAs='" + concept_uri +  "'"

In [ ]:
curl_str = curl_command(URL)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL)
data = json.loads(response.text)

df = pd.json_normalize(data, record_path=['collections'])
df[['id', 'title']]

Get more details about the GCMD concept:

In [ ]:
response = requests.get(concept_uri)
# response.text
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ',newl='')
md("```xml\n" + xmlstr + "\n```\n")

## Collection properties

### Collection identification

In [ ]:
URL = URL_LANDING_PAGE + 'collections/' + 'TropForest'
# URL = URL_LANDING_PAGE + 'collections/' + COLLECTION_ID1

In [ ]:
curl_str = curl_command(URL)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL)
data = json.loads(response.text)
jstr = json.dumps(data, indent=3)
md("```json\n" + jstr + "\n```\n")

In [ ]:


# use stac_client class for STAC collection
c = Collection.from_dict(data)
print("id\t\t:", c.id)
print("title\t\t:", c.title)
print("description\t:", c.description)
print("keywords\t:", c.keywords)
print("spatial extent\t:", c.extent.spatial)
print("temporal extent\t:", c.extent.temporal)
# print("providers\t:", c.providers)
# c

The collection id (`id`) is to be used as `collections` parameter for a corresponding STAC item (granule) search.  It can also be used in the `ids` parameter when searching collections by identifier.

### Collection DOI

Not all collections have a digital object identifier assigned.  if they do, then it is available as `sci:doi` property.  This value can be used for searching collections by DOI.  Collections with DOI, typically also contain a link with rel="cite-as" referring to their landing page.

In [ ]:
try: 
    print(data['sci:doi'])
except:
    print("Not available")

### Collection geometry

Geometry information for a collection is included in the JSON response at the path `$.extent.spatial`.

In [ ]:
data['extent']['spatial']

### Collection temporal extent

The JSON response element provides temporal information for a collection, i.e. the start time and end time at the path `$.extent.temporal`.  The end time may be absent indicating that the collection is not completed.

In [ ]:
try: 
    print(data['extent']['temporal'])
except:
    print("Not available")

```{index} double: STAC API ; collection assets
```
```{index} double: collection assets ; ISO19139 
```
```{index} double: collection assets ; ISO19139-2 
```
```{index} double: collection assets ; ISO19115-3 
```
```{index} double: collection assets ; ISO19157-2 
```
```{index} double: collection assets ; DIF 10 
```
```{index} double: collection assets ; GeoDCAT-AP 
```

### Collection assets

Collections provide access to a dictionary with `assets`.  The `roles` attribute indicates the purpose of the asset. The `href` attribute provides the URL to access the asset.  Collection assets may include `thumbnail` (when available), `search` interfaces, and various `metadata` formats. 

The table below list some frequently used `metadata` formats and their corresponding media type (`type`).

| Format                   | type |   
| --------                   | --------- | 
| [ISO19139](https://www.iso.org/standard/32557.html)        | application/vnd.iso.19139+xml |  
| [ISO19139-2](https://www.iso.org/standard/57104.html)      | application/vnd.iso.19139-2+xml | 
| [ISO19115-3](https://www.iso.org/standard/32579.html)      | application/vnd.iso.19115-3+xml | 
| [ISO19157-2](https://www.iso.org/standard/66197.html)      | application/vnd.iso.19157-2+xml | 

In [ ]:
URL = URL_LANDING_PAGE + 'collections/' + COLLECTION_ID1

In [ ]:
response = requests.get(URL)
data = json.loads(response.text)

# Show assets of the collection (GeoJSON)
jstr = json.dumps(data['assets'], indent=3)
md("```json\n" + jstr + "\n```\n")

In [ ]:
# Display assets belonging to the collection
c = Collection.from_dict(data)
assets = c.assets
df = pd.DataFrame(columns=['roles', 'title', 'type'])
for key in assets:
    ndf = pd.DataFrame({ 
            'roles': assets[key].roles, 
            'type': assets[key].media_type, 
            'title': assets[key].title, 
            # 'href': assets[key].href  
        }, index = [0])
    df = pd.concat([df, ndf], ignore_index=True)
df

```{index} double: STAC API ; collection links
```

### Collection links

Collections provide access to additional resources via `links`.  The `rel` attribute indicates the purpose of the resource. The `href` attribute provides the URL to access the resource.  Collection assets may include `thumbnail` (when available), `search` interfaces, and various `metadata` formats. 


In [ ]:
# Display links belonging to the collection
links = c.links
df = pd.DataFrame(columns=['rel', 'title', 'type'])
for link in links:    
    ndf = pd.DataFrame({ 'rel': link.rel,'type': link.media_type, 'title': link.title }, index = [0])
    df = pd.concat([df, ndf], ignore_index=True)
df

Of particular importance is the link providing access to the list of filter criteria available for granule search within this collection.  This link provides access to a Queryables object in JSON Schema format.

In [ ]:
links = c.get_links(rel = 'http://www.opengis.net/def/rel/ogc/1.0/queryables', media_type = 'application/schema+json' )
links[0].href

In [ ]:
response = requests.get(links[0].href)   
data = json.loads(response.text)    
df = pd.DataFrame(data['properties'].items(),columns=['key','value'])
df['type'] = df.apply(lambda row : row[1]['type'], axis = 1)
df['format'] = df.apply(lambda row : row[1]['format'] if 'format' in row[1] else '-' , axis = 1)
df.drop('value',axis=1).sort_values(by=['key'])

```{index} double: pystac_client ; granule search
```
```{index} double: STAC API ; granule search
```
## Granule Search

```{index} double: STAC API ; landing page
```

### Access landing page

The landing page provides access to collections (rel="`data`"), child catalogs (rel="`child`") and the STAC item search endpoint (rel="`search`").

In [ ]:
from pystac_client import Client 

api = Client.open(URL_LANDING_PAGE) 
# show as a dictionary
api.to_dict()

The STAC granule search endpoint can be found in the landing page (rel="search").  When performing searches, the collections to be searched are specified using their `id`.  You can find the `id` by browsing the catalogue/collection hierarchy or via a collection search.

In [ ]:
# Get STAC granule search link to be used.
print("Search link\t:", api.get_search_link())

```{index} double: STAC API ; intersects
```
```{index} double: STAC API ; GET
```

### Search by geometry

Collections support granule search with the `intersects` [[RD11]](#RD11) search parameter.

```{index} double: pystac_client ; intersects
```

**Example: 2.1**  
>  Search granules by geometry {intersects} [[RD11]](#RD11) and `GET` method.  Geometry parameter can be provided as dictionary or string.

In [ ]:
# See https://pystac-client.readthedocs.io/en/stable/usage.html
#     https://pystac-client.readthedocs.io/en/stable/tutorials.html
#     https://pystac-client.readthedocs.io/en/latest/tutorials/item-search-intersects.html

aoi_as_dict: Dict[str, Any] = {
    "type": "Polygon",
    "coordinates": [
      [
        [
          14.90,
          37.700
        ],
        [
          14.90,
          37.780
        ],
        [
          14.99,
          37.780
        ],
        [
          14.99,
          37.700
        ],
        [
          14.90,
          37.700
        ]
      ]
    ]
}

from pystac_client import Client 
api = Client.open(URL_LANDING_PAGE) 

results = api.search(
    method = 'GET',         
    max_items = 2,
    collections=[COLLECTION_ID1],
    # intersects = json.dumps(aoi_as_dict), 
    intersects = aoi_as_dict,
    datetime=['2015-01-01T00:00:00Z', '2022-01-02T00:00:00Z']
)

In [ ]:
curl_str = curl_command(results.url_with_parameters(), "GET")
md("```shell\n" + curl_str + "\n```\n")

```{index} double: response element ; numberMatched (STAC)
```
The total number of results available is reported in the `numberMatched` property.

In [ ]:
print(f"{results.matched()} items found.")

In [ ]:
# Show search response (GeoJSON)
data = results.get_all_items_as_dict()
jstr = json.dumps(data, indent=3)
md("```json\n" + jstr + "\n```\n")

```{index} double: STAC API ; POST
```

**Example: 2.2**  
>  Search granules by geometry {intersects} [[RD11]](#RD11) and `POST` method.  Geometry parameter can be provided as dictionary or string.

In [ ]:
# same request with POST
from pystac_client import Client 
api = Client.open(URL_LANDING_PAGE) 

results = api.search(
    method = 'POST',        
    max_items = 2,
    collections=[COLLECTION_ID1],
    # intersects = json.dumps(aoi_as_dict), 
    intersects = aoi_as_dict,
    datetime=['2015-01-01T00:00:00Z', '2022-01-02T00:00:00Z']
)

In [ ]:
curl_str = curl_command(results.url_with_parameters(), "POST")
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results.matched()} items found.")

```{index} double: STAC API ; bbox
```

### Search by bounding box

```{index} double: pystac_client ; bbox
```

The geometry parameter can be provided as Python list or tuple.

**Example: 2.3**  
>  Search granules by bounding box {bbox} list [[RD11]](#RD11).  Geometry parameter is provided as Python list.

In [ ]:
from pystac_client import Client
api = Client.open(URL_LANDING_PAGE) 

results = api.search(
    method = 'GET',   
    max_items=10,
    collections=[COLLECTION_ID1],
    bbox = [14.90, 37.700, 14.99, 37.780], # Mount Etna
    # datetime=['2015-01-01T00:00:00Z', '2022-01-02T00:00:00Z']
)

Same request using `curl`.

In [ ]:
curl_str = curl_command(results.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results.matched()} items found.")

In [ ]:
display_previews(results)

In [ ]:
display_gdf_plot(results)

In [ ]:
display_map(results)

In [ ]:
display_date_distribution(results)

**Example: 2.4**  
>  Search granules by bounding box {bbox} [[RD11]](#RD11).  Geometry parameter is provided as Python tuple.

In [ ]:
# x, y = (14.95, 37.74)   # Center point of query (Mount Etna)
x, y = (4.38, 51.25)   # Center point of query (Antwerp harbour) 

r = 0.1
box = (x - r, y - r, x + r, y + r)

from pystac_client import Client 
api = Client.open(URL_LANDING_PAGE) 

results = api.search(
    method = 'GET',   
    max_items=10,
    collections=[COLLECTION_ID1],
    bbox = box
)

Same request using `curl`.

In [ ]:
curl_str = curl_command(results.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results.matched()} items found.")

In [ ]:
display_previews(results)

In [ ]:
display_gdf_plot(results)

In [ ]:
display_map(results)

**Example: 2.5**  
>  Search granules by bounding box (`bbox`) and generate density map. 

In [ ]:
collection_id = COLLECTION_ID1

def get_results(bbox):
    x, y, x2, y2 = bbox
    results = api.search(
            method = 'GET',   
            max_items=1, 
            bbox = [x, y, x2, y2], 
            collections=[collection_id]
    ) 
    return results.matched()

collection_size = get_results([-180, -90, 180, 90])

In [ ]:
n_rows = 36
n_columns = 72

dy = 180.0 / n_rows
dx = 360.0 / n_columns
shape = (n_rows, n_columns)
Z = np.zeros(shape)

bboxes = []
for col in range(n_columns):
    for row in range(n_rows):
        x = col * dx - 180.0
        y = row * dy - 90.0
        bboxes.append((x, y, x+dx, y+dy))

In [ ]:
%%time
executor = ThreadPoolExecutor(max_workers=16)

results = executor.map(get_results, bboxes)

for col in range(n_columns):
    for row in range(n_rows):
        count = next(results)
        Z[row, col] = count

In [ ]:
print(f'Display number of granules as density map of {n_rows} rows ({dy}°) by {n_columns} columns ({dx}°).')

In [ ]:
# Get world map data from Geopandas
worldmap = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

# Create axes and plot world map
fig, ax = plt.subplots(figsize=(12, 6))
worldmap.plot(color="lightgrey", ax=ax)

side_x = np.linspace(-180, +180, n_columns)
side_y = np.linspace(-90, +90, n_rows)
X, Y = np.meshgrid(side_x, side_y)
# Z was computed before.
plt.pcolormesh(X, Y, Z, shading='auto', alpha=0.6)
plt.colorbar(label='Granules')

# Create axis limits and title
plt.xlim([-180, 180])
plt.ylim([-90, 90])

plt.title("Density Plot - " + collection_id + " (size: "+str(collection_size)+")")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

```{index} double: STAC API ; datetime
```
```{index} double: pystac_client ; datetime
```

### Search by temporal extent

**Example: 2.6**  
>  Search granules by date range (datetime) [[RD01]](#RD01).  

In [ ]:
from pystac_client import Client 
api = Client.open(URL_LANDING_PAGE) 

results = api.search(
    method = 'GET',   
    max_items = 50,
    collections=[COLLECTION_ID1],
    datetime=['2019-01-01T00:00:00Z', '2019-12-02T00:00:00Z']
)

In [ ]:
curl_str = curl_command(results.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
display_date_distribution(results)

**Example: 2.7**  
>  Search granules by open-ended date range (datetime) [[RD01]](#RD01).  

In [ ]:
from pystac_client import Client 
api = Client.open(URL_LANDING_PAGE) 

results = api.search(
    method = 'GET',   
    max_items = 50,
    collections=[COLLECTION_ID1],
    datetime=['2021-12-01T00:00:00Z', None]
)

In [ ]:
curl_str = curl_command(results.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results.matched()} items found.")

In [ ]:
# keep id of first granule for future use below.
items = list(results.get_items())
granule_id1 = items[0].id

In [ ]:
display_date_distribution(results)

In [ ]:
display_value_distribution(results, 'sar:product_type')

```{index} double: STAC API ; ids
```

### Search by identifier


```{index} double: pystac_client ; ids
```
**Example: 2.8**  
>  Search granule by identifier (ids) [[RD01]](#RD01).  

In [ ]:
granule_id1

In [ ]:
from pystac_client import Client 
api = Client.open(URL_LANDING_PAGE) 

results = api.search(
    method = 'GET',   
    collections=[COLLECTION_ID1],
    ids=[granule_id1]
)

In [ ]:
curl_str = curl_command(results.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
items = list(results.get_items())
print(f"{results.matched()} items found.")
assert results.matched() == 1

# Convert STAC items into data frame
stac_json = results.get_all_items_as_dict()
gdf = gpd.GeoDataFrame.from_features(stac_json, "epsg:4326")
gdf.transpose()

**Example: 2.9**  
>  Search granule by identifier (`ids`) [[RD01]](#RD01) without specifying collection. 

In [ ]:
from pystac_client import Client 
api = Client.open(URL_LANDING_PAGE) 

results = api.search(
    method = 'GET',   
    ids=[granule_id1]
)

Same request with `curl`.

In [ ]:
curl_str = curl_command(results.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results.matched()} items found.")
assert results.matched() == 1

```{index} double: STAC API ; filter
```

### Search with filter

```{index} double: pystac_client ; filter
```

**Example: 2.10**  
>  Search granules with filter {filter} [[RD01]](#RD01).  Available filters are advertised in `Queryables` object at /collections/{id}/queryables.

In [ ]:
from pystac_client import Client 
api = Client.open(URL_LANDING_PAGE) 

results = api.search(
    method = 'GET',   
    max_items=10,
    collections=[COLLECTION_ID1],
    bbox = [14.90, 37.700, 14.99, 37.780], # Mount Etna
    datetime=['2015-01-01T00:00:00Z', '2022-01-02T00:00:00Z'],
    filter="productType='CHR_MO2_1P' and instrument='CHRIS'"
)

Same request with `curl`.

In [ ]:
curl_str = curl_command(results.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results.matched()} items found.")

In [ ]:
display_previews(results)

In [ ]:
display_gdf_plot(results)

In [ ]:
display_map(results)

In [ ]:
display_value_distribution(results, 'sar:product_type')

### Search by cloud cover

**Example: 2.11**  
>  Search granules by cloudcover (`filter` and `cloudCover`) [[RD01]](#RD01).  Available filters are advertised in `Queryables` object at /collections/{id}/queryables.

In [ ]:
from pystac_client import Client 
api = Client.open(URL_LANDING_PAGE)  

results = api.search(
    method = 'GET',   
    max_items=50,
    collections=[COLLECTION_ID3_CLOUDS],
    filter="cloudCover < 10"    
)

In [ ]:
curl_str = curl_command(results.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results.matched()} items found.")

In [ ]:
# Display cloud-cover values as histogram to show that range is taken into account
stac_json = results.get_all_items_as_dict()
gdf = gpd.GeoDataFrame.from_features(stac_json)
try:
  _ = gdf[['title','eo:cloud_cover']].hist()
except:
  print("eo:cloud_cover information is not available.")

In [ ]:
# fails if properties are not in the metadata.
try:
  # _ = gdf[['view:sun_elevation','view:incidence_angle','view:sun_azimuth']].plot.hist(alpha=0.7)
  _ = gdf[['view:sun_elevation','view:sun_azimuth']].plot.hist(alpha=0.7)
except:
  print("acquisition angle information is not available.")

In [ ]:
# gdf

In [ ]:
# display_value_distribution(results, 'sat:orbit_state')
display_value_distribution(results, 'sar:product_type')

### Search multiple collections

**Example: 2.12**  
>  Search granules in multiple collections {collections} [[RD01]](#RD01).  

In [ ]:
from pystac_client import Client 
api = Client.open(URL_LANDING_PAGE) 

results = api.search(
    method = 'GET',   
    max_items=10,
    collections=[COLLECTION_ID2_CLOUDS, COLLECTION_ID1],
    bbox = [13.90, 36.700, 15.99, 38.780], # Mount Etna (large)
)

In [ ]:
curl_str = curl_command(results.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results.matched()} items found.")

**Example: 2.13**  
>  Search granules in multiple collections {collections} [[RD01]](#RD01) using `POST`. 

In [ ]:
from pystac_client import Client 
api = Client.open(URL_LANDING_PAGE) 

results = api.search(
    method = 'POST',    
    max_items=50,
    collections=[COLLECTION_ID2_CLOUDS, COLLECTION_ID1],
    bbox = [13.90, 36.700, 15.99, 38.780] # Mount Etna (large)
)

In [ ]:
curl_str = curl_command(results.url_with_parameters(),'POST')
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results.matched()} items found.")

## Granule properties

Granules are returned via `item` links in the Catalog or Collection objects, or via the STAC API (Feature).
 An item is a GeoJSON `Feature` and the encoding is derived from the original OGC 17-003r2 encoding 
 according to a [documented mapping](https://github.com/stac-utils/stac-crosswalks/tree/master/OGC_17-003r2).
     
 The properties available include attributes from STAC extensions as well:    
  
 * [Item fields](https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md#item-fields) 
 * [Common metadata elements](https://github.com/radiantearth/stac-spec/blob/master/item-spec/common-metadata.md) 
 * [EO Extension](https://github.com/stac-extensions/eo)
 * [SAR Extension](https://github.com/stac-extensions/sar)
 * [SAT Extension](https://github.com/stac-extensions/sat)
 * [Scientific Extension](https://github.com/stac-extensions/scientific)   
 * [Version Extension](https://github.com/stac-extensions/version)
 * [View Extension](https://github.com/stac-extensions/view)
 * [Projection Extension](https://github.com/stac-extensions/projection)
 * [Timestamps Extension](https://github.com/stac-extensions/timestamps)
 * [Landsat Extension](https://landsat.usgs.gov/stac/landsat-extension/schema.json)   



```{index} double: pystac_client ; assets
```
```{index} double: STAC API ; assets 
```
```{index} double: STAC API ; thumbnail 
```
```{index} double: STAC API ; data 
```
```{index} double: STAC API ; metadata 
```
```{index} double: assets ; OGC 10-157r4 
```
```{index} double: assets ; OGC 17-003r2 
```

### Assets

Granules provide access to a dictionary with `assets`.  The `roles` attribute indicates the purpose of the asset. The `href` attribute provides the URL to access the asset.  Granule assets include `thumbnail` (when available), a `data` download link (equivalent to the rel=`enclosure`), and various `metadata` formats.

The table below list some frequently used `metadata` formats and their corresponding media type (`type`).

| Format                   | type |   
| --------                   | --------- | 
| [ISO19139](https://www.iso.org/standard/32557.html)        | application/vnd.iso.19139+xml |  
| [ISO19139-2](https://www.iso.org/standard/57104.html)      | application/vnd.iso.19139-2+xml | 
| [ISO19115-3](https://www.iso.org/standard/32579.html)      | application/vnd.iso.19115-3+xml | 
| [OGC 10-157r4](https://docs.opengeospatial.org/is/10-157r4/10-157r4.html)  | application/gml+xml;profile=http://www.opengis.net/spec/EOMPOM/1.1  |
| [OGC 17-003r2](https://docs.opengeospatial.org/is/17-003r2/17-003r2.html)  | application/geo+json;profile=http://www.opengis.net/spec/eo-geojson/1.0  |

In [ ]:
# Show assets of first search result (GeoJSON)
data = results.get_all_items_as_dict()
jstr = json.dumps(data['features'][1]['assets'], indent=3)
md("```json\n" + jstr + "\n```\n")

In [ ]:
df = pd.DataFrame(columns=['roles', 'title', 'type'])
    
# Display assets belonging to first item in results
for item in results.items():
    assets = item.assets
    for key in assets:     
        ndf = pd.DataFrame({ 
            'roles': assets[key].roles, 
            'type': assets[key].media_type, 
            'title': assets[key].title, 
            # 'href': assets[key].href  
        }, index = [0])
        df = pd.concat([df, ndf], ignore_index=True)
    
    break
df

## Advanced topics

```{index} double: STAC API ; conformsTo
```

### Conformance classes

The conformance classes supported by the STAC interface are advertised in the `conformsTo` property of the landing page.

In [ ]:
response = requests.get(URL_LANDING_PAGE)

data = json.loads(response.text)
jstr = json.dumps(data['conformsTo'], indent=3)
md("```json\n" + jstr + "\n```\n")

### Additional search parameters


In [ ]:
md(f'Additional search parameters beyond the STAC search parameters can be used to filter collection search results. The available parameters for collection search are advertised at {URL_LANDING_PAGE + "collections/queryables"} and represented as a JSON Schema.')

In [ ]:
URL_QUERYABLES = URL_LANDING_PAGE + 'collections/queryables'

In [ ]:
curl_str = curl_command(URL_QUERYABLES)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL_QUERYABLES)   
data = json.loads(response.text)    
df = pd.DataFrame(data['properties'].items(),columns=['key','value'])
df['type'] = df.apply(lambda row : row[1]['type'], axis = 1)
df['format'] = df.apply(lambda row : row[1]['format'] if 'format' in row[1] else '-' , axis = 1)
df.drop('value',axis=1).sort_values(by=['key'])

In [ ]:
jstr = json.dumps(data, indent=3)
md("```json\n" + jstr + "\n```\n")

Additional search parameters beyond the STAC search parameters can be used to filter granule search results.  The available parameters for granule search are advertised for each individual collection and represented as a JSON Schema.

In [ ]:
URL_COLLECTION_QUERYABLES = URL_LANDING_PAGE + 'collections/' + COLLECTION_ID1 + '/queryables'

md(f"For example, the collection `{COLLECTION_ID1}`, advertises its search parameters \
at {URL_COLLECTION_QUERYABLES} in JSON Schema format. Therefore, the following parameters can be used within a filter expression.")

Get filter parameters for granule search

In [ ]:
curl_str = curl_command(URL_COLLECTION_QUERYABLES)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL_COLLECTION_QUERYABLES)   
data = json.loads(response.text)    
df = pd.DataFrame(data['properties'].items(),columns=['key','value'])
df['type'] = df.apply(lambda row : row[1]['type'], axis = 1)
df['format'] = df.apply(lambda row : row[1]['format'] if 'format' in row[1] else '-' , axis = 1)
df.drop('value',axis=1).sort_values(by=['key'])

In [ ]:
jstr = json.dumps(data, indent=3)
md("```json\n" + jstr + "\n```\n")

### CQL filter expressions

```{index} double: STAC API ; cql-text
```

The STAC interface supports the `filter` parameter and filter expressions in `cql-text` filter format at the following endpoints:

- /collections
- /collections/{collection-id}/items
- /search

At the `/search` endpoint, it is required that a single collection can be determined from the `collections` or `ids` parameter.  The queryables allowed in the filter expression are then identical to the ones at the corresponding `/collections/{collection-id}/items/queryables` endpoint.  `filter` cannot be used at the `/search` endpoint when `collections` contains 0 or more than 1 collection identifiers.

Filter expressions are to be expressed with the Text encoding of the Basic Common Query Language (Basic CQL2-Text) [[RD22]](#RD22).
See the [OGC API Features "Conformance class Filter"](conformance-class-filter) section for CQL2 examples.

**Example: 8.1**  
>  CQL Filter for collection search with logical operators (and, or).

In [ ]:
filter = "platform = 'Envisat' and ( instrument = 'MERIS' or instrument = 'MIPAS' ) and organisationName = 'ESA/ESRIN'"
params = { 'filter': filter } 
URL = f'{URL_LANDING_PAGE}collections?{urllib.parse.urlencode(params)}'

In [ ]:
curl_str = curl_command(URL)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL)
data = json.loads(response.text)
df = pd.json_normalize(data, record_path=['collections'])
df[['id', 'title']]

**Example: 8.2**  
>  CQL filter for granule search with comparison operators.  Search granules with cloudCover between 10 and 15%. 

In [ ]:
from pystac_client import Client 
api = Client.open(URL_LANDING_PAGE)  

results = api.search(
    method = 'GET',   
    max_items = 30,
    collections = [COLLECTION_ID3_CLOUDS],
    filter = "cloudCover >= 10 and cloudCover < 15"   
)

In [ ]:
curl_str = curl_command(results.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results.matched()} items found.")

In [ ]:
# Display cloud-cover values as histogram to show that range is taken into account
stac_json = results.get_all_items_as_dict()
gdf = gpd.GeoDataFrame.from_features(stac_json)
try:
  _ = gdf[['title','eo:cloud_cover']].hist()
except:
  print("eo:cloud_cover information is not available.")

## Further Reading

| **ID**  | **Title** | 
| -------- | --------- | 
| `RD11` <a name="RD11"></a> | [STAC API - Item Search](https://github.com/radiantearth/stac-api-spec/tree/main/item-search) |
| `RD12` <a name="RD12"></a> | [STAC API - Filter Extension](https://github.com/stac-api-extensions/filter) |
| `RD13` <a name="RD13"></a> | [STAC Catalog Specification](https://github.com/radiantearth/stac-spec/blob/master/catalog-spec/catalog-spec.md) | 
| `RD14` <a name="RD14"></a> | [STAC Collection Specification](https://github.com/radiantearth/stac-spec/blob/master/collection-spec/collection-spec.md) | 
| `RD15` <a name="RD15"></a>| [STAC API Specification](https://github.com/radiantearth/stac-api-spec)  | 
| `RD16` <a name="RD16"></a> | [STAC Item Specification](https://github.com/radiantearth/stac-spec/tree/master/item-spec)   | 
| `RD17` <a name="RD17"></a> | [PySTAC Documentation](https://pystac.readthedocs.io/en/stable/) | 
| `RD18` <a name="RD18"></a> | [PySTAC Client Usage](https://pystac-client.readthedocs.io/en/stable/usage.html) | 
| `RD19` <a name="RD19"></a> | [ODC STAC - Plot STAC Items on a map ](https://odc-stac.readthedocs.io/en/latest/notebooks/stac-load-e84-aws.html#Plot-STAC-Items-on-a-Map) | 
| `RD20` <a name="RD20"></a> | [OGC17-069r3, OGC API - Features - Part 1: Core](https://docs.opengeospatial.org/is/17-069r3/17-069r3.html) | 
| `RD21` <a name="RD21"></a> | [OGC17-079r1, OGC API - Features - Part 3: Filtering](https://docs.opengeospatial.org/DRAFTS/19-079r1.html)  | 
| `RD22` <a name="RD22"></a> | [OGC21-065, Common Query Language (CQL2)](https://docs.ogc.org/DRAFTS/21-065.html)  | 
| `RD23` <a name="RD23"></a> | [RFC 7946 - The GeoJSON Format](https://datatracker.ietf.org/doc/html/rfc7946) | 
| `RD24` <a name="RD24"></a>| [JSON Schema: A Media Type for Describing JSON Documents, draft-handrews-json-schema-02](https://datatracker.ietf.org/doc/html/draft-handrews-json-schema-02) |
| `RD25` <a name="RD25"></a>| [STAC API - Collection Search](https://github.com/stac-api-extensions/collection-search) |
| `RD27` <a name="RD27"></a> | [Intake-STAC Documentation](https://intake-stac.readthedocs.io/en/stable/) | 






